In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [2]:
def FDDLM_Model(data):
    model = keras.Sequential(name="FDDLM_Sequential")
    print(data.shape[1:])
    model.add( layers.Input(shape=data.shape[1:]) )
    model.add( layers.Dense(300, activation='relu', name="Dense_1"))
    model.add( layers.Dense(100, activation='relu', name="Dense_2"))
    model.add( layers.Dense(5, activation='softmax', name='output'))
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy','AUC','F1Score','Recall'])
    print(model.summary())
    return model

In [3]:
X = np.load("../COPD_dataset/k-fold-validation/X2_data-002.npy")
y = X[:, -1]

y = y.astype(np.int64)
X = X[:,:-1]
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [4]:
def get_model_performance(X, y, model_func, k_splits = 5,epochs=100):
    kfold = KFold(n_splits= k_splits, shuffle=False)
    full_history =  []
    confusion_matrices= []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        one_hot_train_labels = to_categorical(y_train)
        one_hot_test_labels = to_categorical(y_test)
        del y_train,y_test
        model = model_func(X_train)
        history = model.fit(X_train, one_hot_train_labels, epochs=epochs,batch_size=64,validation_data=(X_test, one_hot_test_labels))
        full_history.append(history)
        del history
        pred = model.predict(X_test)
        pred = np.argmax(pred, axis=1)
        confusion_matrix = metrics.confusion_matrix(pred, one_hot_test_labels)
        confusion_matrices.append(confusion_matrix)
        del confustion_matrix

    return full_history, confusion_matrices

In [5]:
fddlm_history, fddlm_confusion_matrices = get_model_performance(model_func= FDDLM_Model, X= X, y= y, epochs=100)

(72000,)


Model: "FDDLM_Sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense_1 (Dense)                 │ (None, 300)            │    21,600,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_2 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,630,905 (82.52 MB)

 Trainable params: 21,630,905 (82.52 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 107ms/step - AUC: 0.5448 - F1Score: 0.2332 - Recall: 0.2509 - accuracy: 0.2663 - loss: 38.5769 - val_AUC: 0.6438 - val_F1Score: 0.1999 - val_Recall: 0.3844 - val_accuracy: 0.3844 - val_loss: 7.7612
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step - AUC: 0.6597 - F1Score: 0.3359 - Recall: 0.3688 - accuracy: 0.3729 - loss: 5.3058 - val_AUC: 0.6917 - val_F1Score: 0.2168 - val_Recall: 0.3179 - val_accuracy: 0.3202 - val_loss: 3.8252
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - AUC: 0.7254 - F1Score: 0.4057 - Recall: 0.4332 - accuracy: 0.4458 - loss: 2.8458 - val_AUC: 0.7859 - val_F1Score: 0.4367 - val_Recall: 0.6054 - val_accuracy: 0.6054 - val_loss: 1.8672
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - AUC: 0.7654 - F1Score: 0.4250 - Recall: 0.4457 - accuracy: 0.4750 - loss: 2.0205 - val_AUC: 0.7943 - val_F1Score: 0.2844 - val_Recall: 0.4961 - val_accuracy: 0.5073 - val_loss: 1.6121
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s

Epoch 31/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - AUC: 0.9739 - F1Score: 0.7343 - Recall: 0.7678 - accuracy: 0.7706 - loss: 0.3997 - val_AUC: 0.9777 - val_F1Score: 0.6881 - val_Recall: 0.7847 - val_accuracy: 0.7858 - val_loss: 0.3449
Epoch 32/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - AUC: 0.9704 - F1Score: 0.7391 - Recall: 0.7777 - accuracy: 0.7781 - loss: 0.4228 - val_AUC: 0.7007 - val_F1Score: 0.2397 - val_Recall: 0.4081 - val_accuracy: 0.4081 - val_loss: 4.9834
Epoch 33/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - AUC: 0.9533 - F1Score: 0.7204 - Recall: 0.7565 - accuracy: 0.7585 - loss: 0.6988 - val_AUC: 0.9712 - val_F1Score: 0.6562 - val_Recall: 0.7497 - val_accuracy: 0.7711 - val_loss: 0.4704
Epoch 34/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - AUC: 0.9653 - F1Score: 0.7058 - Recall: 0.7546 - accuracy: 0.7573 - loss: 0.4859 - val_AUC: 0.9725 - val_F1Score: 0.6719 - val_Recall: 0.7655 - val_accuracy: 0.7678 - val_loss: 0.3805
Epoch 35/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s

KeyboardInterrupt: 